In [1]:
!pip install datasets -i https://pypi.tuna.tsinghua.edu.cn/simple

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
from datasets import load_dataset

dataset = load_dataset("rootsautomation/RICO-ScreenAnnotation-f")

/apps/miniconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from PIL import Image
import os
i=0
for data in dataset['train']:
    # 创建目录（如果不存在）
    save_path = '/hpc2hdd/home/yliu364/Qwen-VL/' + data['file_name']
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    # 保存图片到特定文件中，以'file_name'命名
    data['image'].save(save_path)
    i=i+1

In [55]:
type(dataset['train'][0]['screen_annotation'])

str

In [3]:
import re
import json

def process_annotation(annotation):
    # 正则表达式匹配括号中的内容
    cleaned_annotation = re.sub(r'\([^()]*\)', '', annotation)
    pattern = r'(\w+(\s+\w+)*)\s*\[([^\[\]]*)\]'
    matches = re.findall(pattern, cleaned_annotation)

    processed_annotations = []
    for match in matches:
        # 提取图标和参数值
        icon = match[0]
        params = match[2].split()
        try:
            x1, y1, x2, y2 = map(float, params[:4])
            x1 = x1*1000
            x2 = x2*1000
            y1 = y1*1000
            y2 = y2*1000
        except ValueError:
            # 如果无法转换为浮点数，跳过当前注释
            continue
        # 构建输出格式
        processed_annotations.append({
            "icon": icon,
            "para": f"<box>({x1:.2f},{y1:.2f}),( {x2:.2f},{y2:.2f})</box>"
        })

    return processed_annotations

In [59]:
dataset['train'][0]

{'screen_id': '39382',
 'screen_annotation': 'TEXT Currency code [0.08 0.92 0.08 0.11], LIST_ITEM [0.05 0.96 0.12 0.18] (RADIO_BUTTON [0.09 0.16 0.13 0.17], LABEL USD [0.21 0.31 0.13 0.16]), LIST_ITEM [0.04 0.96 0.18 0.25] (RADIO_BUTTON [0.10 0.16 0.20 0.23], LABEL UYU [0.20 0.31 0.20 0.23]), LIST_ITEM [0.04 0.96 0.25 0.31] (RADIO_BUTTON [0.10 0.16 0.26 0.30], LABEL UZS [0.20 0.30 0.26 0.29]), LIST_ITEM [0.04 0.96 0.31 0.38] (RADIO_BUTTON [0.10 0.16 0.33 0.36], LABEL VEB [0.21 0.30 0.33 0.36]), LIST_ITEM [0.04 0.96 0.38 0.44] (RADIO_BUTTON [0.10 0.16 0.39 0.43], LABEL VND [0.21 0.31 0.40 0.43]), LIST_ITEM [0.04 0.96 0.44 0.51] (RADIO_BUTTON [0.10 0.16 0.46 0.49], LABEL VUV [0.21 0.31 0.46 0.49]), LIST_ITEM [0.04 0.96 0.51 0.57] (RADIO_BUTTON [0.10 0.16 0.52 0.56], LABEL WST [0.21 0.31 0.53 0.56]), LIST_ITEM [0.04 0.96 0.58 0.64] (RADIO_BUTTON [0.10 0.16 0.59 0.62], LABEL XAF [0.20 0.30 0.59 0.62]), LIST_ITEM [0.04 0.96 0.64 0.70] (RADIO_BUTTON [0.10 0.16 0.66 0.69], LABEL XCD [0.20 0.3

In [5]:
new_json_data = []
for data in dataset['train']:
  # try:
    screen_id = data['screen_id']
    screen_annotation = data['screen_annotation']
    file_name = data['file_name']

    processed_annotations = process_annotation(screen_annotation)

    conversations = []
    for i in range(0, len(processed_annotations), 3):
        for annotation in processed_annotations[i:i+3]:
            icon = annotation['icon']
            para = annotation['para']

            # 添加用户对话
            user_dialog = {
              "from": "user",
              "value": f"Picture: <img>{file_name}</img>\n Give the relative coordinates of {icon} with respect to the image"
            }
            conversations.append(user_dialog)

            # 添加助手对话
            assistant_dialog = {
              "from": "assistant",
              "value": para
            }
            conversations.append(assistant_dialog)
        if len(conversations) >= 6:
            break

    # 构建新的数据项
    new_data = {
        "id": screen_id,
        "conversations": conversations
    }
    new_json_data.append(new_data)
  # except TypeError:
  #   continue

In [6]:
len(new_json_data)

15548

In [7]:
new_json_data_3000 = new_json_data[:3050]

In [8]:
items_to_remove = list(range(13, 17)) + list(range(364, 369))
new_json_data_3000 = [item for i, item in enumerate(new_json_data_3000) if i not in items_to_remove]

In [10]:
import json
with open('/hpc2hdd/home/yliu364/Qwen-VL/new_data_500.json', 'w') as f:
    # 序列化数据并写入文件
    json.dump(new_json_data_3000, f)  # 使用indent参数使输出的JSON更易读

In [65]:
new_json_data_test = new_json_data[4550:5050]
import json
with open('/hpc2hdd/home/yliu364/Qwen-VL/new_data_test.json', 'w') as f:
    # 序列化数据并写入文件
    json.dump(new_json_data_test, f)  # 使用indent参数使输出的JSON更易读